In [1]:
import pandas as pd
import numpy as np
import torch as pt
import cv2 as cv

import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import LFWPairs
from torch.utils.data import DataLoader
from torchvision import transforms

In [ ]:
training = LFWPairs('./data/train', split='train', download=True, transform=transforms.Compose([
    transforms.Resize((220, 220)),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
]))

validating = LFWPairs('./data/validate', download=True, transform=transforms.Compose([
    transforms.Resize((220, 220)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
]))

testing = LFWPairs('./data/test', split='test', download=True, transform=transforms.Compose([
    transforms.Resize((220, 220)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
]))

In [ ]:
training_set = DataLoader(training, batch_size=64, shuffle=True)
validation_set = DataLoader(validating, batch_size=64, shuffle=True)
testing_set = DataLoader(testing, batch_size=64, shuffle=True)

In [8]:
class ContrastiveLoss(nn.Module): 
    
    def __init__(self, margin=2.0):
        super().__init__()
        self.margin = margin
    
    def forward(self, input1, input2, label):
        pairwise_dist = F.pairwise_distance(input1, input2)
        distance = (1 - label) * torch.pow(pairwise_dist, 2) + \
            (label) * torch.pow(torch.clamp(margin - pairwise_dist, min=0), 2)
        return distance
        

In [15]:
class SiameseNetwork(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.distance = ContrastiveLoss()
        
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(7, 7), stride=2, padding=3),
            nn.MaxPool2d(kernel_size=(3, 3), stride=2, padding=1),
            nn.BatchNorm2d(64),
            
            nn.Conv2d(64, 64, kernel_size=(1, 1)),
            nn.Conv2d(64, 192, kernel_size=(3, 3)),
            nn.BatchNorm2d(192),
            nn.MaxPool2d(kernel_size=(3, 3), stride=2, padding=4),
            
            nn.Conv2d(192, 192, kernel_size=(1, 1)),
            nn.Conv2d(192, 384, kernel_size=(3, 3)),
            nn.MaxPool2d(kernel_size=(3, 3), stride=2, padding=1),
            
            nn.Conv2d(384, 384, kernel_size=(1, 1)),
            nn.Conv2d(384, 256, kernel_size=(3, 3)),
            
            nn.Conv2d(256, 256, kernel_size=(1, 1)),
            nn.Conv2d(256, 256, kernel_size=(3, 3)),
            
            nn.Conv2d(256, 256, kernel_size=(1, 1)),
            nn.Conv2d(256, 256, kernel_size=(3, 3)),
            nn.MaxPool2d(kernel_size=(3, 3), stride=2, padding=7)
        )
        
        self.fc1 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(12544, 4096),
            nn.ReLU(inplace=True),
            
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            
            nn.Linear(4096, 128),
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(128, 1),
            nn.Sigmoid()   
        )
    
    def forward_once(self, img):
        return self.fc1(self.cnn(img))
    
    def get_embeddings(self, img):
        return self.forward_once(img)
    
    def forward(self, img1, img2, label):
        out1 = self.forward_once(img1)
        out2 = self.forward_once(img2)
        dist = self.distance(out1, out2, label)
        return self.fc2(dist)